### Load libraries

In [1]:
import os
import pandas as pd

# import function files
import sys
file = 'etl_functions.py'
sys.path.insert(0,os.path.dirname(os.path.abspath(file)))
import etl_functions

### Define variables

In [ ]:
config_load_file_name = 'config_load.yml'
item_DATABASE_CREDENTIALS = 'DATABASE_CREDENTIALS'
item_NEW_TABLE_COLUMNS = 'NEW_TABLE_COLUMNS'
item_TABLE_NAME = 'TABLE_NAME'
item_NEW_COLUMNS_IF_TABLE_EXISTS = 'NEW_COLUMNS_IF_TABLE_EXISTS'
item_FILE_TO_UPLOAD = 'FILE_TO_UPLOAD'
item_PRIMARY_KEY_COLUMN = 'PRIMARY_KEY_COLUMN'

### 3. Load

#### 3.1 Create the database connection
Create the database connection using the information from the _DATABASE_CREDENTIALS_ block of the **config_et.yml file**.

In [ ]:
connection, cursor = etl_functions.database_connection(config_load_file_name, item_DATABASE_CREDENTIALS)

#### 3.2 Create table 

In [ ]:
# create PostgreSQL statement 
create_table_sql_statement = etl_functions.sql_statement_create_table_if_not_exists(config_load_file_name, item_NEW_TABLE_COLUMNS, item_TABLE_NAME)

# run the PostgreSQL statement
etl_functions.execute_sql_statement(cursor, connection, [create_table_sql_statement])

# get the PostgreSQL table information
etl_functions.print_columns(cursor, config_load_file_name, item_TABLE_NAME)

#### 3.3 Add new column 

In [ ]:
# create PostgreSQL statement
add_column_sql_statement =  etl_functions.sql_statement_add_column_if_table_exists(config_load_file_name, item_NEW_COLUMNS_IF_TABLE_EXISTS, item_TABLE_NAME)

# run the PostgreSQL statement
etl_functions.execute_sql_statement(cursor, connection, [add_column_sql_statement])

# get the PostgreSQL table information
etl_functions.print_columns(cursor, config_load_file_name, item_TABLE_NAME)

#### 3.4 Load data

In [ ]:
etl_functions.insert_dataframe_to_database(connection, cursor, config_load_file_name, item_FILE_TO_UPLOAD, 
                                           item_TABLE_NAME, item_PRIMARY_KEY_COLUMN)

#### 3.5 Close the database conection

In [ ]:
cursor.close()
connection.close()

### 4. Some useful SQL statements 

#### Create the database connection
Create the database connection using the information from the _DATABASE_CREDENTIALS_ block of the **config_et.yml file**.

In [ ]:
connection, cursor = etl_functions.database_connection(config_load_file_name, item_DATABASE_CREDENTIALS)

#### Describe table
Describe the structure of a table specified in the **table_name** variable.

In [ ]:
table_name = 'write_your_table_name'

etl_functions.describe_table(cursor, table_name)

#### Querying a Table
In the following link, you can find the syntax of the [SQL](https://www.w3schools.com/sql/default.asp) statements that you can use for querying a table. 

[https://www.postgresql.org/docs/current/tutorial-select.html](https://www.postgresql.org/docs/current/tutorial-select.html)

In [ ]:
# create the sql statement
sql_statement = 'SELECT * FROM write_your_table_name'

# execute SQL query
cursor.execute(sql_statement)

query_results = cursor.fetchall()

# create the dataframe
colnames = [desc[0] for desc in cursor.description]
    
data_frame = pd.DataFrame(data = query_results, columns = colnames)

data_frame

#### Delete column
Delete the column specified in the **column_name** variable from the table set in the **table_name** variable. 

**CAUTION! Be careful before dropping a column. Deleting a column will result in the loss of complete information stored in that column!**

In [ ]:
table_name = 'write_your_table_name'
column_name = 'write_your_column_name'

# sql =  'ALTER TABLE ' +  table_name + ' DROP COLUMN ' + column_name 

# cursor.execute(sql)

#### Delete table
Delete the table specified in the **table_name** variable. 

**CAUTION! Be careful before dropping a table. Deleting a table will result in the loss of complete information stored in the table!**

In [ ]:
table_name = 'write_your_table_name'

#delete_table_sql = 'DROP table ' + table_name 

#cursor.execute(delete_table_sql)

#### Close the database conection

In [ ]:
cursor.close()
connection.close()